This notebook is a part of motokimura's baseline solution for the [Solafune Identifying Deforestation Drivers competition](https://solafune.com/competitions/68ad4759-4686-4bb3-94b8-7063f755b43d?menu=about&tab=overview).
See https://github.com/motokimura/solafune_deforestation_baseline for the complete code.

> cf. @solafune (https://solafune.com) Use for any purpose other than participation in the competition or commercial use is prohibited. If you would like to use them for any of the above purposes, please contact us.

### Description

**By running this notebook, you will achieve a score of around 0.227 on the public leaderboard.**

This notebook trains a U-Net model for 4-class segmentation (`grassland_shrubland`, `logging`, `mining`, and `plantation`) and generates a submission JSON file for the evaluation images from the output from the U-Net model.

The submission JSON file is saved to `data/submission.json`.

Before running this notebook, you have to run `generate_masks.ipynb` to generate `.npy` files used for training
(`generate_masks.ipynb` is available from https://github.com/motokimura/solafune_deforestation_baseline).

### Requirements

#### Datasets

Organize the dataset as follows:

```
data/
├── evaluation_images/
│   ├── evaluation_0.tif
│   ├── evaluation_1.tif
│   ├── evaluation_2.tif
│   ├── ...
├── train_images/
│   ├── train_0.tif
│   ├── train_1.tif
│   ├── train_2.tif
│   ├── ...
├── train_masks/
│   ├── train_0.npy
│   ├── train_1.npy
│   ├── train_2.npy
│   ├── ...
```

`evaluation_images` and `train_images` can be downloaded from the competition page.

`train_masks` can be generated by running `generate_masks.ipynb` available from https://github.com/motokimura/solafune_deforestation_baseline.

#### Libraries

Please install the python packages imported the cell below.


In [1]:
import json
import os

from pathlib import Path

import albumentations as albu  # tested with 1.4.24
import imagecodecs
import numpy as np  # tested with 1.26.4
import pandas as pd
import pytorch_lightning as pl  # tested with 2.5.0.post0
import segmentation_models_pytorch as smp  # tested with 0.3.4
import sklearn
import tensorboard
import tifffile
import timm  # tested with 0.9.7
import torch  # tested with 2.5.1

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from rasterio import features
from shapely.geometry import Polygon, shape
from skimage import measure
from solafune_tools.metrics import F1_Metrics
from timm.optim import create_optimizer_v2
from timm.scheduler import create_scheduler_v2
from tqdm import tqdm

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed_everything(42)

Seed set to 42


42

In [3]:
data_root = Path("./data")

class_names = ["grassland_shrubland", "logging", "mining", "plantation"]

epochs = 200

### Define dataset class to load images and masks for training and validation

In [4]:
def load_mask(mask_path):
    mask = np.load(mask_path)  # (4, H, W), uint8
    assert mask.shape == (4, 1024, 1024)
    mask = mask.transpose(1, 2, 0)  # (H, W, 4)
    return mask.astype(np.float32) / 255.0  # normalize to [0, 1]


def load_image(image_path):
    image = tifffile.imread(image_path)  # (H, W, 12), float64
    assert image.shape == (1024, 1024, 12)
    image = np.nan_to_num(image)  # replace NaN with 0
    return image.astype(np.float32)


def normalize_image(image):
    # mean of train images
    mean = np.array(
        [
            285.8190561180765,
            327.22091430696577,
            552.9305957826701,
            392.1575148484924,
            914.3138803812591,
            2346.1184507500043,
            2884.4831706095824,
            2886.442429854111,
            3176.7501338557763,
            3156.934442092072,
            1727.1940075511282,
            848.573373995044,
        ],
        dtype=np.float32
    )

    # std of train images
    std = np.array(
        [
            216.44975668759372,
            269.8880248304874,
            309.92790753407064,
            397.45655590699,
            400.22078920482215,
            630.3269651264278,
            789.8006920468097,
            810.4773696969773,
            852.9031432100967,
            807.5976198303886,
            631.7808113929271,
            502.66788721341396,
        ],
        dtype=np.float32
    )
    
    mean = mean.reshape(12, 1, 1)
    std = std.reshape(12, 1, 1)

    return (image - mean) / std


class TrainValDataset(torch.utils.data.Dataset):
    def __init__(self, data_root, sample_indices, augmentations=None):
        self.image_paths, self.mask_paths = [], []
        for i in sample_indices:
            self.image_paths.append(data_root / "train_images" / f"train_{i}.tif")
            self.mask_paths.append(data_root / "train_masks" / f"train_{i}.npy")
        self.augmentations = augmentations

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        sample = {
            "image": load_image(self.image_paths[idx]),
            "mask": load_mask(self.mask_paths[idx]),
        }

        if self.augmentations is not None:
            sample = self.augmentations(**sample)

        sample["image"] = sample["image"].transpose(2, 0, 1)  # (12, H, W)
        sample["mask"] = sample["mask"].transpose(2, 0, 1)  # (4, H, W)

        sample["image"] = normalize_image(sample["image"])

        # add metadata
        sample["image_path"] = str(self.image_paths[idx])
        sample["mask_path"] = str(self.mask_paths[idx])

        return sample

### Define U-Net model using pytorch-lightning

In [5]:
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # prepare segmentation model
        self.model = m = smp.create_model(
            arch='unet',
            encoder_name='tu-tf_efficientnetv2_s',  # use `tf_efficientnetv2_s` from timm
            encoder_weights='imagenet',  # always starts from imagenet pre-trained weight
            in_channels=12,
            classes=4,
        )

        # prepare loss functions
        self.dice_loss_fn = smp.losses.DiceLoss(mode=smp.losses.MULTILABEL_MODE, from_logits=True)
        self.bce_loss_fn = smp.losses.SoftBCEWithLogitsLoss(smooth_factor=0.0)

        self.training_step_outputs = []
        self.validation_step_outputs = []
    
    def forward(self, image):
        # assuming image is already normalized
        return self.model(image)  # logits

    def shared_step(self, batch, stage):
        image = batch["image"]
        mask = batch["mask"]

        logits_mask = self.forward(image)

        loss = self.dice_loss_fn(logits_mask, mask) + self.bce_loss_fn(logits_mask, mask)

        # count tp, fp, fn, tn for each class to compute validation metrics at the end of epoch
        thresh = 0.5
        prob_mask = logits_mask.sigmoid()
        tp, fp, fn, tn = smp.metrics.get_stats(
            (prob_mask > thresh).long(),
            mask.long(),
            mode=smp.losses.MULTILABEL_MODE,
        )  # each of tp, fp, fn, tn is a tensor of shape (batch_size, num_classes) and of type long

        output = {
            "loss": loss.detach().cpu(),
            "tp": tp.detach().cpu(),
            "fp": fp.detach().cpu(),
            "fn": fn.detach().cpu(),
            "tn": tn.detach().cpu(),
        }
        if stage == "train":
            self.training_step_outputs.append(output)
        else:
            self.validation_step_outputs.append(output)

        return loss

    def training_step(self, batch, batch_idx):
        return self.shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self.shared_step(batch, "val")

    def shared_epoch_end(self, outputs, stage):
        def log(name, tensor, prog_bar=False):
            self.log(f"{stage}/{name}", tensor.to(self.device), sync_dist=True, prog_bar=prog_bar)

        # aggregate loss
        loss = torch.stack([x["loss"] for x in outputs]).mean()
        log("loss", loss, prog_bar=True)

        # aggregate tp, fp, fn, tn to compose IoU score for each class
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])

        iou_scores = {}
        for i, class_name in enumerate(class_names):
            iou_scores[class_name] = smp.metrics.iou_score(tp[:, i], fp[:, i], fn[:, i], tn[:, i], reduction="macro-imagewise")
            log(f"iou/{class_name}", iou_scores[class_name], prog_bar=False)

        iou_avg = torch.stack([v for v in iou_scores.values()]).mean()
        log("iou", iou_avg, prog_bar=True)
    
    def on_train_epoch_end(self):
        self.shared_epoch_end(self.training_step_outputs, "train")
        self.training_step_outputs.clear()

    def on_validation_epoch_end(self):
        self.shared_epoch_end(self.validation_step_outputs, "val")
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        # optimizer
        optimizer = create_optimizer_v2(
            self.parameters(),
            opt="adamw",
            lr=1e-4,
            weight_decay=1e-2,
            filter_bias_and_bn=True,  # filter out bias and batchnorm from weight decay
        )

        # lr scheduler
        scheduler, _ = create_scheduler_v2(
            optimizer,
            sched="cosine",
            num_epochs=epochs,
            min_lr=0.0,
            warmup_lr=1e-5,
            warmup_epochs=0,
            warmup_prefix=False,
            step_on_epochs=True,
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch",
            },
        }

    def lr_scheduler_step(self, scheduler, metric):
        # workaround for timm's scheduler:
        # https://github.com/Lightning-AI/lightning/issues/5555#issuecomment-1065894281
        scheduler.step(epoch=self.current_epoch)  # timm's scheduler need the epoch value

### Prepare trainer of pytorch-lightning

In [6]:
train_output_dir = data_root / "training_result"

# split train_images into train-set and val-set
sample_indices = list(range(176))  # train_0.tif to train_175.tif
train_indices, val_indices = sklearn.model_selection.train_test_split(sample_indices, test_size=0.2, random_state=42)

# augmentations applied only to train-set
augmentations = albu.Compose(
    [
        # shift, scale, and rotate
        albu.ShiftScaleRotate(
            p=0.5,
            shift_limit=0.0625,
            scale_limit=0.1,
            rotate_limit=15,
            border_mode = 0,  # constant border
            value = 0,
            mask_value = 0,
            interpolation = 2,  # bicubic
        ),
        # random crop
        albu.RandomCrop(
            p=1,
            width=512,
            height=512,
        ),
        # flip, transpose, and rotate90
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.Transpose(p=0.5),
        albu.RandomRotate90(p=0.5),
    ]
)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(
    TrainValDataset(
        data_root,
        train_indices,
        augmentations=augmentations,
    ),
    batch_size=16,
    num_workers=8,
    shuffle=True,
)

val_loader = torch.utils.data.DataLoader(
    TrainValDataset(
        data_root,
        val_indices,
        augmentations=None,
    ),
    batch_size=4,
    num_workers=8,
    shuffle=False,
)

# prepare trainer
trainer = Trainer(
    max_epochs=epochs,
    callbacks = [
        # save model with best validation IoU score
        ModelCheckpoint(
            dirpath=train_output_dir,
            filename="best_iou_05",
            save_weights_only=True,
            save_top_k=1,
            monitor="val/iou",
            mode="max",
            save_last=False,
        ),
        LearningRateMonitor(logging_interval="step"),
    ],
    logger=[TensorBoardLogger(train_output_dir, name=None)],
    precision="16-mixed",
    deterministic=True,
    benchmark=False,
    sync_batchnorm=False,
    check_val_every_n_epoch=5,
    default_root_dir=os.getcwd(),
    accelerator="gpu",
    devices=[0,],
    strategy="ddp_notebook",
    log_every_n_steps=5,
)

# prepare model
model = Model()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/tf_efficientnetv2_s.in21k_ft_in1k)
INFO:timm.models._hub:[timm/tf_efficientnetv2_s.in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Converted input conv conv_stem pretrained weights from 3 to 12 channel(s)


### Start training!

With the default setting, 10 GB of GPU memory is required. To reduce the memory usage, you can decrease the batch size.

The trained model is saved as `data/training_result/best_iou_05.ckpt`.

Tensorboard logs are also saved under `data/training_result/version_xx`.

**The execution often does not finish even after reaching 200 epochs. In that case, you can stop the execution manually and just proceed to the next cell (do not restart the notebook!).**

In [7]:
# start training
trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

/opt/conda/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /workspace/data/training_result exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type                  | Params | Mode 
----------------

Epoch 199: 100%|██████████| 9/9 [00:13<00:00,  0.65it/s, v_num=0, train/loss=0.402, train/iou=0.781, val/loss=0.714, val/iou=0.534]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 9/9 [00:13<00:00,  0.64it/s, v_num=0, train/loss=0.402, train/iou=0.781, val/loss=0.714, val/iou=0.534]



Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

### Compute the evaluation metric for the validation set

Before predicting the evaluation images, let's predict the validation set and compute the evaluation metric.

This may be useful to check if the model is trained properly and to tune the parameters for post-processing (e.g., score threshold, etc.).

In [8]:
def run_inference(model, loader, pred_output_dir):
    pred_output_dir = Path(pred_output_dir)
    pred_output_dir.mkdir(exist_ok=True, parents=True)

    for batch in tqdm(loader):
        img = batch["image"].cuda()

        with torch.no_grad():
            logits_mask = model(img)
            prob_mask = logits_mask.sigmoid()

        # save prob mask as numpy array
        for i in range(img.size(0)):
            file_name = os.path.basename(batch["image_path"][i])
            prob_mask_i = prob_mask[i].cpu().numpy()  # (4, 1024, 1024)

            np.save(
                pred_output_dir / file_name.replace(".tif", ".npy"),
                prob_mask_i.astype(np.float16),
            )

In [9]:
# load best checkpoint and run inference on val-set

del model

model = Model()
model.load_state_dict(torch.load(train_output_dir / "best_iou_05.ckpt")["state_dict"])
model = model.cuda()
model.eval()

val_pred_dir = data_root / "val_preds"
run_inference(model, val_loader, val_pred_dir)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/tf_efficientnetv2_s.in21k_ft_in1k)
INFO:timm.models._hub:[timm/tf_efficientnetv2_s.in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Converted input conv conv_stem pretrained weights from 3 to 12 channel(s)
/tmp/ipykernel_851513/2901745631.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 

`detect_polygons()` below extracts isolated areas as polygons from the predicted mask.

The point is `min_area` parameter to filter out small polygons. Small polygons are often false positives which largely decrease the evaluation metric.

In [10]:
def detect_polygons(pred_dir, score_thresh, min_area):
    pred_dir = Path(pred_dir)
    pred_paths = list(pred_dir.glob("*.npy"))
    pred_paths = sorted(pred_paths)

    polygons_all_imgs = {}
    for pred_path in tqdm(pred_paths):
        polygons_all_classes = {}

        mask = np.load(pred_path)  # (4, 1024, 1024)
        for i, class_name in enumerate(class_names):
            label = measure.label(mask[i] > score_thresh, connectivity=2, background=0).astype(np.uint8)

            polygons = []
            for p, value in features.shapes(label, label):
                p = shape(p).buffer(0.5)
                if p.area >= min_area:
                    p = p.simplify(tolerance=0.5)
                    polygons.append(p)

            polygons_all_classes[class_name] = polygons

        polygons_all_imgs[pred_path.name.replace(".npy", ".tif")] = polygons_all_classes

    return polygons_all_imgs

In [11]:
val_pred_polygons = detect_polygons(val_pred_dir, score_thresh=0.5, min_area=10000)

100%|██████████| 36/36 [00:03<00:00, 10.76it/s]


In [12]:
# just load truth polygons from `train_annotations.json` downloaded from the competition page

with open(data_root / "train_annotations.json", "r") as f:
    raw_annotations = json.load(f)

truth_polygons: dict[str, dict[str, list[Polygon]]] = {}  # file_name -> class_name -> polygons
for fn in tqdm([f"train_{i}.tif" for i in range(176)]):
    ann: dict[str, list[list[float]]] = {}  # class_name -> polygons
    for class_name in class_names:
        ann[class_name] = []

    for tmp_img in raw_annotations["images"]:
        if tmp_img["file_name"] == fn:
            for tmp_ann in tmp_img["annotations"]:
                poly = tmp_ann["segmentation"]
                # convert [x1, y1, x2, y2, ..., xn, yn] to [(x1, y1), (x2, y2), ..., (xn, yn)]
                new_poly = Polygon([(poly[i], poly[i + 1]) for i in range(0, len(poly), 2)]).buffer(0)
                ann[tmp_ann["class"]].append(new_poly)

    truth_polygons[fn] = ann

  0%|          | 0/176 [00:00<?, ?it/s]

100%|██████████| 176/176 [00:00<00:00, 1557.78it/s]


In [13]:
# compute evaluation metric for validation set

metric = F1_Metrics()
val_f1_scores = {}

for idx in val_indices:
    fn = f"train_{idx}.tif"
    val_f1_scores[fn] = {}
    for class_name in class_names:
        pred_polys = val_pred_polygons[fn][class_name]
        truth_polys = truth_polygons[fn][class_name]
        f1_score, _, _ = metric.compute_f1(pred_polys, truth_polys)
        val_f1_scores[fn][class_name] = f1_score

val_f1_df = pd.DataFrame(val_f1_scores).T

val_f1_avg = val_f1_df.mean().mean()  # average of all classes and all images
print(f"average f1 score: {val_f1_avg}")

print()
print("f1 scores:")
val_f1_df

average f1 score: 0.08739841623613553

f1 scores:


,grassland_shrubland,logging,mining,plantation
train_19.tif,0.000000,0.0,0.000000,0.222222
train_45.tif,0.000000,0.0,0.000000,0.000000
train_139.tif,0.000000,0.0,0.000000,0.222222
train_30.tif,0.500000,0.0,0.000000,0.000000
train_67.tif,0.000000,0.0,0.142857,0.000000
train_16.tif,0.400000,0.0,0.000000,0.000000
train_119.tif,0.000000,0.0,0.000000,0.000000
train_172.tif,0.000000,0.0,0.000000,0.000000
train_109.tif,0.000000,0.0,0.000000,0.285714
train_140.tif,0.000000,0.0,0.000000,0.666667


### Predict the evaluation images and generate a submission JSON file

As already done with the validation set, predict the evaluation images and generate a submission JSON file.

The submission JSON file is saved as `data/submission.json`.

In [14]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data_root):
        self.image_paths = []
        for i in range(118):  # evaluation_0.tif to evaluation_117.tif
            self.image_paths.append(data_root / "evaluation_images" / f"evaluation_{i}.tif")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        sample = {
            "image": load_image(self.image_paths[idx]),
        }

        sample["image"] = sample["image"].transpose(2, 0, 1)  # (12, H, W)
        sample["image"] = normalize_image(sample["image"])

        # add metadata
        sample["image_path"] = str(self.image_paths[idx])

        return sample

In [15]:
test_loader = torch.utils.data.DataLoader(
    TestDataset(data_root),
    batch_size=4,
    num_workers=8,
    shuffle=False,
)

test_pred_dir = data_root / "test_preds"
run_inference(model, test_loader, test_pred_dir)

100%|██████████| 30/30 [00:09<00:00,  3.32it/s]


In [16]:
test_pred_polygons = detect_polygons(test_pred_dir, score_thresh=0.5, min_area=10000)

submission_save_path = data_root / "submission.json"

images = []
for img_id in range(118):  # evaluation_0.tif to evaluation_117.tif
    annotations = []
    for class_name in class_names:
        for poly in test_pred_polygons[f"evaluation_{img_id}.tif"][class_name]:
            seg: list[float] = []  # [x0, y0, x1, y1, ..., xN, yN]
            for xy in poly.exterior.coords:
                seg.extend(xy)

            annotations.append({"class": class_name, "segmentation": seg})

    images.append({"file_name": f"evaluation_{img_id}.tif", "annotations": annotations})

with open(submission_save_path, "w", encoding="utf-8") as f:
    json.dump({"images": images}, f, indent=4)

100%|██████████| 118/118 [00:10<00:00, 10.98it/s]
